# Key Identity Verification APIs

-------

This lab provides hands on experience with the key Amazon Rekognition APIs for identity verification. 

## Introduction 

-------

In-person user identity verification is slow to scale, costly, and high friction for users. Machine learning powered facial biometrics can enable online user identity verification. Amazon Rekognition offers pre-trained facial recognition and analysis capabilities that you can quickly add to your user onboarding and authentication work flows to verify opted-in users' identity online. 

In this notebook, we'll use the Amazon Rekgonition's key APIs for Identity Verification. After running this notebook you should be able to use the following APIs:

- DetectFaces: DetectFaces detects the 100 largest faces in the image. 
- CompareFaces: Compares a face in the source input image with each of the 100 largest faces detected in the target input image.
- CreateCollection: Creates a searchable index of faces. 
- IndexFaces: Detects faces in the input image and adds them to the specified collection and returns a face id which can be used in subsequent searches. 
- SearchFacesByImage: For a given input image, first detects the largest face in the image, and then searches the specified collection for matching faces.
- SearchFaces: For a given input face ID, searches for matching faces in the collection of the face it belongs to.
- DeleteFaces: Deletes faces from a collection. You specify a collection ID and an array of face IDs to remove from the collection.



In [ ]:
import io
import boto3
import json
from IPython.display import Image as IImage
import pandas as pd

%store -r bucket_name
mySession = boto3.session.Session()
aws_region = mySession.region_name
print("AWS Region: {}".format(aws_region))
print("AWS Bucket: {}".format(bucket_name))


### Setup Clients 
-----
Here we are going to use both S3 and Rekognition APIs  

In [ ]:
s3_client  = boto3.client('s3')
rek_client = boto3.client('rekognition')

#### Display a Face

In [ ]:
## Image of a Face
face_image = "face_6.jpeg"
print(face_image)
display(IImage(url=s3_client.generate_presigned_url('get_object', 
                                                    Params={'Bucket': bucket_name, 
                                                            'Key'   : face_image})))

## DetectFaces API
----

[DetectFaces]("https://docs.aws.amazon.com/rekognition/latest/APIReference/API_DetectFaces.html") detects the 100 largest faces in the image. For each face detected, the operation returns face details. These details include a bounding box of the face, a confidence value (that the bounding box contains a face), and a fixed set of attributes such as facial landmarks (for example, coordinates of eye and mouth), presence of beard, sunglasses, and so on.


<div class="alert alert-info"><strong> FaceDetails </strong>
    
Details of each face found in the image. A [FaceDetail]("https://docs.aws.amazon.com/rekognition/latest/APIReference/API_FaceDetail.html") object contains either the default facial attributes or all facial attributes. The default attributes are BoundingBox, Confidence, Landmarks, Pose, and Quality.

**Take a moment and look at the response from the detect faces api.**

</div>

In [ ]:
response = rek_client.detect_faces(Image={'S3Object':{
    'Bucket': bucket_name,
    'Name'  : face_image}},
    Attributes=['ALL'])

response


## Compare Faces 
------

[CompareFaces]("https://docs.aws.amazon.com/rekognition/latest/APIReference/API_CompareFaces.html")
Compares a face in the source input image with each of the 100 largest faces detected in the target input image. CompareFaces also returns an array of faces that don't match the source image. For each face, it returns a bounding box, confidence value, landmarks, pose details, and quality. The response also returns information about the face in the source image, including the bounding box of the face and confidence value.

<div class="alert alert-info"><strong> FaceMatches </strong>
    
 [FaceMatches]("https://docs.aws.amazon.com/rekognition/latest/APIReference/API_CompareFaces.html#API_CompareFaces_ResponseSyntax") returns an array of faces in the target image that match the source image face. Each CompareFacesMatch object provides the bounding box, the confidence level that the bounding box contains a face, and the similarity score for the face in the bounding box and the face in the source image.


</div>

<div class="alert alert-info"><strong> Key Parameters </strong>

- **QualityFilter:** A filter that specifies a quality bar for how much filtering is done to identify faces. Filtered faces aren't compared. If you specify AUTO, Amazon Rekognition chooses the quality bar. If you specify LOW, MEDIUM, or HIGH, filtering removes all faces that don not meet the chosen quality bar.

- **SimilarityThreshold:** The minimum level of confidence in the face matches that a match must meet to be included in the FaceMatches array. Minimum 0 maximum 100.

**NOTE: take a look at the Quality, Similarity and various landmarks**

</div>

In [ ]:
source_image =  "face_6.jpeg"
target_image =  "face_6_3.jpeg"

response = rek_client.compare_faces(SourceImage={
        'S3Object': {
            'Bucket': bucket_name,
            'Name': source_image,
        }
    },
    TargetImage={
        'S3Object': {
            'Bucket': bucket_name,
            'Name': target_image,
        }
    },
    SimilarityThreshold = 90,
    QualityFilter='HIGH'
)
        
response                   
    

## Creating a Collection
-----

A collection is a searchable index of faces. You create a named collection in an AWS Region and then add faces to the collection using the IndexFaces operation. once the collection is created and faces have been indexed you can search the collection to determine if a face has been indexed in the collection either by image or by face identifier. 


For example, you might create collections, one for each of your application users. A user can then index faces using the IndexFaces operation and persist results in a specific collection. Then, a user can search the collection for faces in the user-specific container.


<div class="alert alert-info"><strong> Creating, Indexing and Searching Collections </strong>
    
The following steps are used to create a collection, index faces and finally search the collection for a face. 

1. [CreateCollection]("https://docs.aws.amazon.com/rekognition/latest/APIReference/API_CreateCollection.html") Creates a collection in an AWS Region.
2. [IndexFaces]("https://docs.aws.amazon.com/rekognition/latest/APIReference/API_IndexFaces.html")Detects faces in the input image and adds them to the specified collection.
3. [SearchFacesByImage]("https://docs.aws.amazon.com/rekognition/latest/APIReference/API_SearchFacesByImage.html") For a given input image, first detects the largest face in the image, and then searches the specified collection for matching faces. The operation compares the features of the input face with faces in the specified collection.
4. [SearchFaces]("https://docs.aws.amazon.com/rekognition/latest/APIReference/API_SearchFacesByImage.html") For a given input face ID, searches for matching faces in the collection the face belongs to. You get a face ID when you add a face to the collection using the IndexFaces operation. The operation compares the features of the input face with faces in the specified collection.
5. [DeleteFaces]("https://docs.aws.amazon.com/rekognition/latest/APIReference/API_DeleteFaces.html")Deletes faces from a collection. You specify a collection ID and an array of face IDs to remove from the collection.


#### Other Collection APIs to try.
------
- [ListCollections]("https://docs.aws.amazon.com/rekognition/latest/APIReference/API_ListCollections.html")Returns list of collection IDs in your account. 
- [ListFaces]("https://docs.aws.amazon.com/rekognition/latest/APIReference/API_ListFaces.html")Returns metadata for faces in the specified collection. This metadata includes information such as the bounding box coordinates, the confidence (that the bounding box contains a face), and face ID.

### Name your Collection 

------

Here you can update the name of your collection

</div>

In [ ]:
## Name your Collection 
collection_name = 'identity-verification-collection'

try:
    # try and create a colleciton, 
    response = rek_client.create_collection(
        CollectionId=collection_name)
except:
    # if it exists, delete it then created it 
    response = rek_client.delete_collection(
        CollectionId=collection_name
    )
    response = rek_client.create_collection(
        CollectionId=collection_name
    )

response


## Indexing Faces 
-----

Here we'll read our collection image mapping file into a panda's dataframe, the image map contains a reference name (used as an external identifier) a reference image (a synthtetically generated face) and four synthetically aged selfie images. 

<div class="alert alert-info"><strong> Next Steps: </strong>
   
- IndexFaces: index a single image into our collection, index multiple faces into our collection 
- SearchFacesByImage : search a collection for a specific image 
- SearchFaces : search a collection for a specific FaceID 
</div>





In [ ]:
# -- read the image map into a pandas dataframe --
obj = s3_client.get_object(Bucket=bucket_name, Key='IDVImageMapping.xlsx')
#df = pd.read_excel(io.BytesIO(obj['Body'].read()))

image_map = pd.read_excel(io.BytesIO(obj['Body'].read()), engine='openpyxl')
image_map.head()

### IndexFaces 
-----

Here we are going to index a single face of Aaron Peirsol - an American former competition swimmer and backstroke specialist who is a former world champion and world record-holder.

<div class="alert alert-info"><strong> Make note of FaceID </strong>
    
Make note of the extracted the FaceID from the response JSON. we'll use this to search for a specific face in the collection. 
</div>

In [ ]:
image_name =  "face_7.jpeg"
image_external_id = "Entity_X_Gentry"

# Index a single face 
response = rek_client.index_faces(
    CollectionId=collection_name,
    Image={
        'S3Object': {
            'Bucket': bucket_name,
            'Name': image_name,
        }
    },
    ExternalImageId=image_external_id,
    DetectionAttributes=[
        'DEFAULT',
    ],
    MaxFaces=1,          # how many faces to detect 
    QualityFilter='AUTO' # apply quality filter 
)

face_id = response['FaceRecords'][0]['Face']['FaceId']
print("-- FaceID : {} --\n".format(face_id))
print(json.dumps(response, indent=3))  



## IndexFaces 
-----

The easy way to index multiple faces is to simply make a list of faces and loop over the list, in this case we are going to use the "reference name" as our external image id and the reference image will be the image we add to our collection. Of course you can use a variety of parallel processing methods to speed up indexing operations, however in this case here we'll keep it simple and just use a for loop. Note we are using about 100 images so this may take a few seconds. 

<div class="alert alert-info"><strong> Make note of ExternalImageID </strong>
    
The ExternalImageID is the ID that you typically use to correlate a face match to your external systems, it can also be used to check identities. In this case our external image id is simply the name of the person in the image.

</div>

In [ ]:

## Index several faces 
dict_of_faces = image_map[["reference_name","reference_image"]].to_dict('records')

for rec in dict_of_faces:
    try:
        response = rek_client.index_faces(
            CollectionId= collection_name,
            Image={
                'S3Object': {
                    'Bucket': bucket_name,
                    'Name': rec["reference_image"],
                }
            },
            ExternalImageId=rec['reference_name'],
            DetectionAttributes=[
                'DEFAULT',
            ],
            MaxFaces=1, # maximum faces detected 
            QualityFilter='AUTO' # apply the quality filter. 
            )
        face_id = response['FaceRecords'][0]['Face']['FaceId']
        print("ImageName: {}, FaceID: {}".format(rec["reference_image"], face_id))
    except:
         print("Failed: ImageName: {}, FaceID: {}".format(rec["reference_image"], face_id))
    

print("indexing complete")
    

## SearchFacesByImage
-----

One of the primary use cases of identity verification is to simply search the collection of faces to see if a particular face already exists in the collection and assess the similarity to the indexed image. 

<div class="alert alert-info"><strong> Things to Note </strong>

- MaxFaces: returns the maximum number of faces with the highest confidence in the match. Minimum value of 1. Maximum value of 4096.
- FaceMatchThreshold: specifies the minimum confidence in the face match to return. For example, don't return any matches where confidence in matches is less than 70%. The default value is 80%. 
- QualityFilter: allows you to filter out detected faces that do not meet a required quality bar. The quality bar is based on a variety of common use cases. Use QualityFilter to set the quality bar for filtering by specifying LOW, MEDIUM, or HIGH.

</div>

In [ ]:
search_image_name =  "face_5_3.jpeg"
response = rek_client.search_faces_by_image(
    CollectionId=collection_name,
    Image={
        'S3Object': {
            'Bucket': bucket_name,
            'Name': search_image_name,
        }
    },
    MaxFaces=1,
    FaceMatchThreshold=90,
    QualityFilter='AUTO'
)
response

external_image_id = response["FaceMatches"][0]["Face"]["ExternalImageId"]
similarity_score =  response["FaceMatches"][0]["Similarity"]
face_id = response["FaceMatches"][0]["Face"]["FaceId"]

print("-- FaceID          : {} ".format(face_id))
print("-- Similarity      : {} ".format(similarity_score))
print("-- ExternalImageID : {} \n".format(external_image_id))

print(json.dumps(response, indent=3))  


## SearchFaces 
-----
For a given input **face ID**, search the collection for  matching faces. You get a face ID when you add a face to the collection using the IndexFaces operation(above) we are simply going to index additional faces below, and snag a couple of face ID. The operation compares the features of the input face with faces in the specified collection. Essentially matching faces within a collection. 



<div class="alert alert-info"><strong> Setup </strong>

- Index additional faces, in this case we'll use our image_map dataframe and take the first 10 selfie_images 
- Make note of a face ID returned from the index 
- SearchFaces for a given *FaceID* and assess the match. 

</div>


In [ ]:
## Index several faces 
dict_of_faces = image_map[["reference_name","selfie_0"]].head(10).to_dict('records')

for rec in dict_of_faces:
  
    response = rek_client.index_faces(
        CollectionId= collection_name,
        Image={
            'S3Object': {
                'Bucket': bucket_name,
                'Name': rec["selfie_0"],
            }
        },
        ExternalImageId=rec['reference_name'],
        DetectionAttributes=[
            'DEFAULT',
        ],
        MaxFaces=1, # maximum faces detected 
        QualityFilter='AUTO' # apply the quality filter. 
        )
    face_id = response['FaceRecords'][0]['Face']['FaceId']
    print("ImageName: {}, FaceID: {}".format(rec["selfie_0"], face_id))
    

print("indexing complete")
    


#####  Snag one of the FaceIDs above 
----
Pass the FaceID above to search_faces, this will identify potential matches within the collection. 

**Select a face id from the results above and enter it below**

In [ ]:

face_id = "" # replace with your own face ID
try:
    response = rek_client.search_faces(
        CollectionId=collection_name,
        FaceId=face_id,
        MaxFaces=10
    )
    print(json.dumps(response, indent=3))  
except:
    print("FaceID not found")

    

## Other Collection APIs
-----

There are several other APIs which we can take advantage of to manage and maintain our collection. 

<div class="alert alert-info"><strong> Add'l APIs </strong>

- ListCollections: Returns list of collection IDs in your account.
- DeleteFaces: Deletes faces from a collection. You specify a collection ID and an array of face IDs to remove from the collection
- ListFaces: Returns meta data for faces in the specified collection. This meta data includes information such as the bounding box coordinates, the confidence (that the bounding box contains a face), and face ID.
- DeleteCollection: Deletes the specified collection. Note that this operation removes all faces in the collection.

</div>


### ListCollections 
-----
ListCollections returns an array of Collection Ids and corresponding FaceModelVersions. This is helpful information to note when upgrading to newer versions of Rekognition's faces model.

In [ ]:
## -- list collections 
response = rek_client.list_collections(
    MaxResults=123
)

response

### DeleteFaces 

------
Deletes faces from a collection. You specify a collection ID and an array of face IDs to remove from the collection. to do this snag two or more FaceIDs from the results above and simply pass them to the DeleteFaces API to remove them from the collection.



In [ ]:
## -- Delete Faces
## Snag a few faceIDs from above. 
response = rek_client.delete_faces(
    CollectionId=collection_name,
    FaceIds=[ "", ""]  # enter faceIDs here
)
response

#### What happens when you pass a faceID that doesn't exist? 
----
Here we are simply going to search for a face id that we deleted, you'll note we get an error that the faceID does not exist

In [ ]:
## Search for a face that doesn't exist 

face_id = "48c04911-4573-439d-9389-bebc0994013a"
try:
    response = rek_client.search_faces(
        CollectionId=collection_name,
        FaceId=face_id,
        MaxFaces=10
    )
except:
    print("FaceID: \'{}\' NOT FOUND IN Collection: \'{}\' ".format(face_id,collection_name ))

### ListFaces 

------
To list the faces in a collection, you can use the ListFaces operation to list the faces in a collection. ListFaces returns metadata for faces in the specified collection. This metadata includes information such as the bounding box coordinates, the confidence (that the bounding box contains a face), and face ID. 



<div class="alert alert-info"><strong> ListFaces API </strong>

- CollectionId : collection identifier
- MaxResults : Maximum number of faces to return. Valid Range: Minimum value of 0. Maximum value of 4096.

note the "NextToken" will allow you to fetch the next set of faces 

</div>

In [ ]:
maxResults = 100
response=rek_client.list_faces(CollectionId=collection_name,
                           MaxResults=maxResults)

print('Faces in collection ' + collection_name)

tokens=True
while tokens:
    faces=response['Faces']

    for face in faces:
        print (face)
    if 'NextToken' in response:
        nextToken=response['NextToken']
        response=rek_client.list_faces(CollectionId=collection_name,
                                   NextToken=nextToken,MaxResults=maxResults)
    else:
        tokens=False

### DeleteCollection 
----- 

As part of our cleanup, we can a delete our collection. This will delete the collection and all the face vectors contained within 

In [ ]:
response = rek_client.delete_collection(
   CollectionId= collection_name
)
response